# **Suryakanta Karan (M22AIE207) m22aie207@iitj.ac.in**



In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1')

#Task1 - resnet18 Which Is A Pre Trained Model

In [ ]:
import torch
import torchvision.models as models
model = models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#Task 2 - Extraction Data

In [ ]:
import os
import urllib.request
import tarfile

# Specify the URL and file name
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2011/VOCtrainval_25-May-2011.tar"
file_name = "VOCtrainval_25-May-2011.tar"

# Specify the path to check for the file
file_path = "/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1/"

# Specify the directory where you want to extract the files
output_directory = os.path.join(file_path, "VOC2011")

# Check if the output_directory already exists
if not os.path.exists(output_directory):
    # Check if the file already exists in the specified path
    if not os.path.exists(os.path.join(file_path, file_name)):
        # If it doesn't exist, download the file
        urllib.request.urlretrieve(url, os.path.join(file_path, file_name))

    # Open the TAR archive
    with tarfile.open(os.path.join(file_path, file_name), 'r') as tar_ref:
        # Extract all the contents to the specified directory
        tar_ref.extractall(output_directory)
else:
    print(f"The directory '{output_directory}' already exists. No need to download and extract.")


The directory '/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1/VOC2011' already exists. No need to download and extract.


#Task 3:

For category A, randomly select 20-50% of the images (making sure to select at least 20% and at most 50%).
For all other categories (category "not A"), randomly select 10% of the images from each category.
The images selected from categories A and "not A" will be used to create the training dataset.

In [ ]:
import random
import os
random.seed(42) # for reproducibility
category_A = "aeroplane"
category_not_A = []
image_names_A = []
image_names_not_A = []
selected_images_not_A = []
# read the file names from the ImageSets folder
with open(f"/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1/VOC2011/TrainVal/VOCdevkit/VOC2011/ImageSets/Main/{category_A}_train.txt", "r") as f:
    for line in f:
        image_name, label = line.split()
        if label == "1": # image belongs to category A
            image_names_A.append(image_name)
        elif label == "-1": # image does not belong to category A
            image_names_not_A.append(image_name)
# randomly select 20%-50% images from category A
num_images_A = len(image_names_A)
num_select_A = random.randint(int(0.2 * num_images_A), int(0.3 * num_images_A))
selected_images_A = random.sample(image_names_A, num_select_A)
# randomly select 10% images from each of the remaining categories
for file in os.listdir("/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1/VOC2011/TrainVal/VOCdevkit/VOC2011/ImageSets/Main"):
    if file.endswith("_train.txt") and file != f"{category_A}_train.txt":
        category = file.split("_")[0]
        category_not_A.append(category)
        image_names = []
        with open(f"/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1/VOC2011/TrainVal/VOCdevkit/VOC2011/ImageSets/Main/{file}", "r") as f:
            for line in f:
                image_name, label = line.split()
                if label == "-1": # image does not belong to category A
                    image_names.append(image_name)
        num_images = len(image_names)
        num_select = int(0.1 * num_images)
        selected_images_not_A.extend(random.sample(image_names, num_select))
# combine the selected images from both categories
training_images = selected_images_A + selected_images_not_A
# shuffle the training images
random.shuffle(training_images)
#print(training_images)


#Task4 - Print the images

In [ ]:
print(training_images)

['2010_004770', '2008_002327', '2011_002606', '2008_008591', '2010_001552', '2009_002089', '2009_003594', '2010_004148', '2010_001148', '2008_004235', '2008_004570', '2010_002216', '2010_002193', '2008_001031', '2008_006857', '2011_003134', '2009_004225', '2010_000863', '2008_008227', '2010_005972', '2008_004239', '2008_001219', '2009_000684', '2008_000742', '2010_004451', '2008_007226', '2010_005807', '2010_005753', '2011_002937', '2011_001272', '2009_000327', '2008_006436', '2011_001967', '2009_002235', '2010_004256', '2009_001251', '2011_003151', '2010_001385', '2009_003818', '2010_000361', '2008_007223', '2008_007660', '2011_000885', '2010_005932', '2008_002202', '2008_002465', '2009_004642', '2009_004643', '2008_007226', '2010_003910', '2009_003369', '2008_000236', '2008_000641', '2008_000970', '2009_002789', '2008_004937', '2010_000302', '2008_005873', '2009_000006', '2008_004776', '2010_002760', '2009_000522', '2010_001973', '2009_001755', '2011_001826', '2008_000471', '2008_002

#Task5 -

Model Evaluation

In [ ]:
import torch
import os
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
import torch
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
# create an empty feature matrix with shape (number of validation images, 1000)
val_features = torch.zeros((5823, 1000))
# create an empty label vector with shape (number of validation images,)
val_labels = torch.zeros((5823,))
# initialize a counter variable
counter = 0
# loop through the validation images and extract features and labels
for i, file in enumerate(os.listdir("/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1/VOC2011/TrainVal/VOCdevkit/VOC2011/ImageSets/Main")):
    if file.endswith("_val.txt"):
        category = file.split("_")[0]
        with open(f"/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1/VOC2011/TrainVal/VOCdevkit/VOC2011/ImageSets/Main/{file}", "r") as f:
            for j, line in enumerate(f):
                image_name, label = line.split()
                # load the image from the JPEGImages folder
                image = Image.open(f"/content/drive/MyDrive/surya/DL_Assignment/Fractal-1_Assignment-1/VOC2011/TrainVal/VOCdevkit/VOC2011/JPEGImages/{image_name}.jpg")
                # apply the transform function to the image
                image = transform(image)
                # add a batch dimension to the image tensor
                image = image.unsqueeze(0)
                # pass the image through the model and get the output of the last fully-connected layer
                #output = model.fc(model(image))
                output = model(image)
                # store the output in the feature matrix using the counter variable as the index
                val_features[counter] = output
                # store the label in the label vector using the counter variable as the index
                if category == category_A:
                    val_labels[counter] = int(label)
                else:
                    val_labels[counter] = -int(label)
                # increment the counter variable by one
                counter += 1
# create a kNN classifier with k=5 and Euclidean distance
knn = KNeighborsClassifier(n_neighbors=5, metric="euclidean")
# fit the classifier on the training features and labels
knn.fit(features, torch.ones((len(training_images),))) # use 1 for category A and 0 for category not A
# predict the labels for the validation features
val_pred = knn.predict(val_features)
# calculate and print the accuracy score
acc = accuracy_score(val_labels, val_pred)
print(f"Accuracy: {acc}")
# calculate and print the confusion matrix
cm = confusion_matrix(val_labels, val_pred)
print(f"Confusion matrix:\n{cm}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# create a figure and an axis object
fig, ax = plt.subplots()
# plot a heatmap of the confusion matrix using seaborn
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
# set the axis labels and title
ax.set_xlabel("Predicted label")
ax.set_ylabel("True label")
ax.set_title("Confusion matrix")
# show the plot
plt.show()



#Observation:

The Google Colab session crashed due to a lack of RAM, even though the GPU was available. The user is advised to consider upgrading to Colab Pro to access more RAM. (Screenshots are attached in the Word document.)

In other words, the user was running a resource-intensive task on Google Colab, and the session crashed because the system ran out of RAM.